# Part 02: Feature Engineering
Building on our exploratory data analysis, we now transition into feature engineering to refine and enrich our dataset, thereby enhancing its predictive power for cyclist traffic in Paris.

In this notebook, we focus on:

- **Refining Data Quality:** 

    Address any remaining data inconsistencies or missing values identified during EDA.

- **Temporal Feature Extraction:** 

    Leverage date and time data to create meaningful features like day of the week, hour, and season.

- **Integrating External Data:** 

    Enrich the dataset with external factors like weather conditions, potentially influential in predicting cyclist traffic.

- **Transforming and Creating Features:** 

    Apply techniques like one-hot encoding and normalization, and develop new features informed by EDA insights.

- **Strategic Feature Selection:** 

    Employ methods to select impactful features, aiming to reduce complexity while maintaining model efficacy.

In [1]:
# installs
#pip install holidays
#pip install vacances-scolaires-france
#pip install lockdowndates

In [2]:
#imports

from pathlib import Path
import os
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow as pa
from sklearn.impute import SimpleImputer, KNNImputer
from IPython.display import display
from IPython import get_ipython
import holidays
from meteostat import Point, Hourly
from vacances_scolaires_france import SchoolHolidayDates
from lockdowndates.core import LockdownDates

plt.style.use('fivethirtyeight')
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=0.8)

In [3]:
#Load the data
bike_df_train = pd.read_parquet(Path("data") / "train.parquet")
bike_df_test = pd.read_parquet(Path("data") / "test.parquet")

## **Feature Extraction**:
In this phase, we extract and refine various features from our dataset to enhance the predictive modeling of bike traffic in Paris. The process involves:

- **Date and Time Extraction**: 

    We dissect the `date` column to extract key temporal components - `year`, `month`, `day`, and `hour`. This breakdown aids in understanding traffic patterns across different time features.

- **Seasonality Identification**: 

    By mapping month to `season` and identifying `weekdays` and French public `holidays`, we capture seasonal and weekly traffic trends. Additionally, we integrate `school holiday` data specific to Paris (Zone C) to account for variations during school breaks.

- **Weather Data Integration**: 

    Utilizing the `meteostat` library, we fetch and clean `weather` data for distinct geographical locations in our dataset. By using the geolocalization coordinates (`latitude` and `longitude`) of each site, we can extract the hourly weather at each of the locations. This step ensures each entry in our dataset is enriched with relevant and timely weather information, which is vital for analyzing its impact on bike traffic.

These enhancements in feature extraction aim to provide a comprehensive and nuanced view of the factors affecting bike traffic in Paris, setting the stage for more accurate and insightful modeling.

In [4]:
bike_df_train['date'].dtype

dtype('<M8[ns]')

We currently only have a column `date` of type `<M8[us]` which is a datetime with microsecond precision. We can take advantage of this date datatype to create more explicit columns of each information regarding a date (e.g. `year`, `month`, `day`, `hour`).

We will go even further and add information regarding whether the day is a `working day`, a `holiday` and also consider the `seasons` for a more general analysis.

In [5]:
# Define a function to map months to seasons
def get_season(month):
    """
    Map a given month to a corresponding season.

    Parameters:
    - month (int): Numeric representation of the month (1 to 12).

    Returns:
    - str: The season associated with the input month. Possible values are 'Spring', 'Summer', 'Fall', or 'Winter'.

    Example:
    >>> get_season(7)
    'Summer'

    Note:
    The function considers the following month ranges for each season:
    - Spring: March (3) to May (5)
    - Summer: June (6) to August (8)
    - Fall: September (9) to November (11)
    - Winter: December (12), January (1), and February (2)
    """
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Fall'
    else:
        return 'Winter' # December, January, February

In [6]:
# Initialize the SchoolHolidayDates object
school_holidays = SchoolHolidayDates()

#Check if a given date is a school holiday
def is_school_holiday(datetime_obj):
    """
    Check if a given date, extracted from a datetime object, corresponds to a school holiday in Zone C (Paris).

    Parameters:
    - datetime_obj (datetime.datetime): Input datetime object.

    Returns:
    - bool: True if the date corresponds to a school holiday, False otherwise.
    """
    # Extracting just the date part from the datetime object
    date_obj = datetime_obj.date()
    return school_holidays.is_holiday_for_zone(date_obj, 'C')  # Paris is Zone C

In [7]:
def encode_dates(df):
    """
    Encode temporal features from a given DataFrame containing a 'date' column.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with a 'date' column of datetime type.

    Returns:
    - pd.DataFrame: DataFrame with additional columns encoding temporal features, including:
        - 'year': Year extracted from the 'date' column.
        - 'month': Month extracted from the 'date' column.
        - 'day': Day of the month extracted from the 'date' column.
        - 'hour': Hour of the day extracted from the 'date' column.
        - 'weekday': Numeric representation of the day of the week (0 for Monday, 6 for Sunday).
        - 'season': Season associated with the 'month' column using the get_season function.
        - 'working_day': Binary indicator for working days (1 for weekdays not in French holidays, 0 otherwise).
        - 'school_holiday': Binary indicator for school holidays using the is_school_holiday function.

    Raises:
    - ValueError: If the input DataFrame does not have a 'date' column or if the 'date' column is not of datetime type.
    """
    if 'date' not in df.columns or not pd.api.types.is_datetime64_any_dtype(df['date']):
        raise ValueError("DataFrame must have a 'date' column of datetime type.")

    X = df.copy()

    # Extracting date components
    X['year'] = X['date'].dt.year
    X['month'] = X['date'].dt.month
    X['day'] = X['date'].dt.day
    X['hour'] = X['date'].dt.hour
    X['weekday'] = X['date'].dt.weekday

    # Adding season based on month
    X['season'] = X['month'].apply(get_season)

    # Identifying working days (weekdays not in French holidays)
    fr_holidays = holidays.France()
    X['working_day'] = ((X['date'].dt.weekday < 5) & ~X['date'].dt.date.isin(fr_holidays)).astype(int)

    # Identifying school holidays
    X['school_holiday'] = X['date'].apply(is_school_holiday).astype(int)

    return X

In [8]:
# Record the start time
start_time = time.time()

bike_df_train = encode_dates(bike_df_train)

# Record the end time
end_time = time.time()

# Calculate the runtime
runtime = round(end_time - start_time, 2)

# Print the runtime
print(f"Runtime: {runtime} seconds")
bike_df_train.head(5)

Runtime: 247.72 seconds


,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,year,month,day,hour,weekday,season,working_day,school_holiday
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,2,1,Fall,1,0
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,2020,9,1,3,1,Fall,1,0
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,2020,9,1,4,1,Fall,1,0
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,2020,9,1,15,1,Fall,1,0
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,2020,9,1,18,1,Fall,1,0


The `weather` can have a great impact on bike traffic so we will also add weather information from `meteostat`.

In [9]:
def get_weather_data(X):
    """
    Retrieve hourly weather data for unique geographical locations and combine it into a Pandas DataFrame.

    Parameters:
    - X (pd.DataFrame): Input DataFrame with columns 'date', 'latitude', and 'longitude' (among others).

    Returns:
    - pd.DataFrame: Add new features on weather data for all unique locations, including columns:
        - 'date': Date and time of the weather observation.
        - 'temperature': Hourly temperature data.
        - 'humidity': Hourly humidity data.
        - 'precipitation': Hourly precipitation data.
        - 'latitude': Constant latitude value for the location.
        - 'longitude': Constant longitude value for the location.

    Raises:
    - ValueError: If the input DataFrame does not contain the required columns.
    """
    if not all(col in X.columns for col in ['date', 'latitude', 'longitude']):
        raise ValueError("DataFrame must have 'date', 'latitude', and 'longitude' columns.")

    unique_locations = X[['latitude', 'longitude']].drop_duplicates()
    weather_data_list = []

    for lat, lon in unique_locations.itertuples(index=False):
        point = Point(lat, lon)
        location_data = Hourly(point, X['date'].min(), X['date'].max()).fetch()
        location_data.reset_index(inplace=True)  # Reset index to make 'date' a column
        location_data.rename(columns={'time': 'date'}, inplace=True)  # Rename 'time' column to 'date'
        location_data['latitude'], location_data['longitude'] = lat, lon
        weather_data_list.append(location_data)

    weather_data = pd.concat(weather_data_list, ignore_index=True)

    # Ensure 'date' column is in the right format if it's not already
    if weather_data['date'].dtype != 'datetime64[ns]':
        weather_data['date'] = pd.to_datetime(weather_data['date'])

    # Merge data
    merged_data = pd.merge(X, weather_data, on=['date', 'latitude', 'longitude'], how='left')
    
    return merged_data

In [10]:
# Record the start time
start_time = time.time()

bike_df_train = get_weather_data(bike_df_train)

# Record the end time
end_time = time.time()

# Calculate the runtime
runtime = round(end_time - start_time, 2)

# Print the runtime
print(f"Runtime: {runtime} seconds")
bike_df_train.head(5)


Runtime: 7.93 seconds


,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,9.4,79.0,0.0,NaN,290.0,3.6,9.3,1019.9,NaN,1.0
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,9.7,81.0,0.0,0.0,310.0,5.4,5.0,1019.8,NaN,1.0
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,9.6,83.0,0.0,NaN,300.0,5.4,7.4,1019.5,NaN,1.0
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,7.8,46.0,0.0,0.0,30.0,14.8,28.0,1017.5,NaN,3.0
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,7.0,46.0,NaN,0.0,40.0,11.2,24.0,1017.6,NaN,4.0


## **Final Data Cleaning**:
The final data cleaning stage is critical in preparing our dataset for accurate and effective modeling. After incorporating external data sources, we now ensure the dataset's cleanliness and readiness for analysis. This involves:

- **Handling Missing Values**: 

    Applying tailored imputation strategies (zero-fill for precipitation, mean-fill for temperature, humidity, wind speed, and KNN imputation for snowfall) to maintain data integrity

- **Removing Duplicates**: 

    Eliminating duplicate entries to prevent skewed analysis

- **Data Type Conversion**: 

    Converting the 'date' column to datetime format and setting it as the index for easier manipulation and analysis

- **Dropping Redundant Columns**: 

    Removing columns that don't contribute to the predictive power of our model, like counter names, raw bike counts, and certain temporal attributes

## Handling Missing Values

In [11]:
# Checking for missing values
missing_values = bike_df_train.isna().sum()

# Show only variables with null values greater than 1
missing_values_greater_than_1 = missing_values[missing_values > 1]
missing_values_greater_than_1

prcp     22428
snow    341809
tsun    496771
dtype: int64

As we can see, there are only missing values regarding the weather data we used that's why it's important to double check after our initial EDA.

We cannot just remove rows that have missing values because we would lose a lot of information. Moreover, it is reasonable to assume that the amount of rain (`prcp`), snow (`snow`), and sunshine (`tsun`) do affect the number of bikes. Therefore, we need to consider what to do with all these missing values.

There are several ways we could handle these missing values.

#### 1. Replace missing values by 0

This strategy is suitable when missing values are interpreted as zero or when you believe that the missing values occur randomly or don't carry significant information. When looking at the amount of rain, snow or sunshine, it makes sense to treat missing values as 0 (as if we don't have any information at those times). Be cautious when applying this strategy, especially if missing values are not distributed randomly. In cases where missing values represent a pattern or specific conditions, replacing them with zero may introduce bias. We would need to see if the missing values are scattered randomly in the dataset or if there are several rows with missing values.

For example, it would be unreasonable to set the amount of rain to be 0 for the whole month of December.

#### 2. Replace missing values by the mean / median

Another strategy is to replace the amounts by either the mean or the median of the distribution. However, a mean value should be used on variable's with a "normal" distribution (no strong skewness, outliers). <br> A median value should be used when the distribution is skewed with many outliers. As in the case of strategy 1, this strategy assumes that the missing values are missing completely at random.

#### 3. Replace missing values by the most frequent value:

This approach is more suitable for categorical variables where the most frequent category can reasonably stand in for the missing values. This strategy may be less appropriate for continuous variables.

#### 4. Use Machine Learning techniques to predict the missing values

We can also use techniques such as KNN Imputer, for imputing missing values in a dataset. Instead of using a single value (like mean, median, or zero), KNN imputation estimates missing values by considering the values of their k-nearest neighbors. However, it adds a lot more computational time.

For `prcp`, setting missing values to 0 seems reasonable since we assume they indicate no precipitation 

In [12]:
bike_df_train['prcp'].fillna(0, inplace=True)

For `tsun`, we believe the amount of sunshine does not influence the number of bikes in Paris. Indeed, whether it is cloudy or sunny, as long as it is not raining or snowing, regular bikers will still take the bike.

In [13]:
bike_df_train.drop(columns="tsun", inplace=True) 

For `snow`, let's investigate the number of missing values in winter:

In [14]:
winter_data = bike_df_train.query('season == "Winter"')

# Check how many entries are there for the Winter season
num_winter_entries = winter_data.shape[0]

# Output the number of Winter entries
print(f"Number of entries during Winter: {num_winter_entries}")

Number of entries during Winter: 120518


In [15]:
# Number of missing amount of rain and snow
winter_data[['snow']].isna().sum()

snow    81858
dtype: int64

We believe that in winter, the weather is more or less stable and that we can estimate the missing amount of snow using `KNNImputer`.

In [16]:
# To save time, we already ran the computation and saved it in a new file called bike_df_imputed
knn_flag = False

if(knn_flag):

    # Apply imputation - Note: This might be computationally expensive for large datasets
    bike_df_train[['snow']] = KNNImputer().fit_transform(bike_df_train[['snow']].to_numpy().reshape(-1, 1))

    # Save the DataFrame with the imputed values in the 'data' directory
    bike_df_train.to_parquet('data/bike_df_imputed.parquet')

In [17]:
# Load the dataset after imputation
bike_df_imputed = pd.read_parquet('data/bike_df_imputed.parquet')

# Check the number of missing values for 'snow' before and after imputation
missing_values_before = bike_df_train['snow'].isnull().sum()
missing_values_after = bike_df_imputed['snow'].isnull().sum()

# Calculate the summary statistics for 'snow' before and after imputation
summary_before = bike_df_train['snow'].describe()
summary_after = bike_df_imputed['snow'].describe()

# Compare the distributions of 'snow' before and after imputation
comparison_summary = pd.DataFrame({'Before Imputation': summary_before, 'After Imputation': summary_after})

# Display the results
print(f"Missing 'snow' values before imputation: {missing_values_before}")
print(f"Missing 'snow' values after imputation: {missing_values_after}")
comparison_summary

Missing 'snow' values before imputation: 341809
Missing 'snow' values after imputation: 0


,Before Imputation,After Imputation
count,155018.000000,496827.000000
mean,0.108375,0.108375
std,1.492652,0.833769
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.108375
75%,0.000000,0.108375
max,40.000000,40.000000


The KNN imputation effectively addressed all missing values in the 'snow' column, reducing them from 341,809 to 0. Post-imputation, the distribution of 'snow' shows a lower standard deviation, indicating reduced variability, while the mean value remains almost unchanged. This suggests a smoothing effect on the data, with potential impacts on model performance that should be evaluated.

In [18]:
# Check for duplicate rows
duplicate_rows = bike_df_imputed.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

# Remove duplicate rows if any
if duplicate_rows > 0:
    bike_df_imputed = bike_df_imputed.drop_duplicates()
    print("Duplicates removed.")
else:
    print("No duplicates found.")

# Check summary statistics to identify any anomalies
bike_df_imputed.describe()

Number of duplicate rows: 0
No duplicates found.


,site_id,bike_count,latitude,longitude,log_bike_count,year,month,day,hour,weekday,...,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco
count,4.968270e+05,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,...,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000,496827.000000
mean,1.053450e+08,60.191475,48.854343,2.345479,3.079917,2020.679846,6.556904,15.458226,11.502730,2.992172,...,12.917013,7.091982,70.013957,0.088281,0.108375,185.039364,11.321216,24.229571,1016.412786,4.032233
std,3.210346e+07,87.590566,0.018554,0.038026,1.659899,0.466536,3.423834,8.851485,6.920936,1.995015,...,6.873227,5.914273,16.055120,0.441823,0.833769,102.795006,5.112426,11.135895,9.478988,3.597789
min,1.000070e+08,0.000000,48.826360,2.265420,0.000000,2020.000000,1.000000,1.000000,0.000000,0.000000,...,-6.400000,-17.100000,1.000000,0.000000,0.000000,0.000000,0.000000,1.800000,972.200000,1.000000
25%,1.000475e+08,5.000000,48.840801,2.314440,1.791759,2020.000000,4.000000,8.000000,6.000000,1.000000,...,7.900000,2.600000,59.000000,0.000000,0.000000,80.000000,7.600000,16.700000,1011.300000,2.000000
50%,1.000562e+08,29.000000,48.852090,2.353870,3.401197,2021.000000,7.000000,15.000000,12.000000,3.000000,...,12.500000,7.400000,73.000000,0.000000,0.108375,200.000000,11.200000,22.200000,1017.900000,3.000000
75%,1.000563e+08,79.000000,48.864610,2.375870,4.382027,2021.000000,9.000000,23.000000,18.000000,5.000000,...,17.800000,11.900000,83.000000,0.000000,0.108375,260.000000,14.800000,31.000000,1022.700000,4.000000
max,3.000147e+08,1302.000000,48.891720,2.409690,7.172425,2021.000000,12.000000,31.000000,23.000000,6.000000,...,34.100000,20.700000,100.000000,16.000000,40.000000,360.000000,40.700000,94.000000,1039.500000,25.000000


In [19]:
# Validate key variables
print("Anomalies in bike_count:", bike_df_imputed[bike_df_imputed['bike_count'] < 0].shape[0])

# Final check for missing values
missing_values = bike_df_imputed.isna().sum()
print("Missing values in each column:")
print(missing_values)

Anomalies in bike_count: 0
Missing values in each column:
counter_id                   0
counter_name                 0
site_id                      0
site_name                    0
bike_count                   0
date                         0
counter_installation_date    0
coordinates                  0
counter_technical_id         0
latitude                     0
longitude                    0
log_bike_count               0
year                         0
month                        0
day                          0
hour                         0
weekday                      0
season                       0
working_day                  0
school_holiday               0
temp                         0
dwpt                         0
rhum                         0
prcp                         0
snow                         0
wdir                         0
wspd                         0
wpgt                         0
pres                         0
coco                         0
dtype: int64

## Functions for Feature Engineering

In this section, we automate all of the feature engineering process for simplicity and avoid redundancy when performing feature engineering on our training and testing sets.

## **Creating new Features**
In this stage, we focus on enhancing our dataset with creatively engineered features, tailored to capture the complex dynamics of bike traffic in Paris. We delve into:

- **Weather Categorization**: 

    We create a 'weather' feature categorizing conditions into Clear, Windy, Rainy, or Snowy, based on deviations from mean values of precipitation, snow, and wind speed

- **Time of Day Analysis**: 

    We categorize hours into Morning, Afternoon, Evening, and Night to reflect daily traffic patterns

- **Rush Hour Identification**: 

    We flag rush hours, considering typical peak traffic times in the mornings and evenings

- **Lockdown in France**:

    Since our data reflects the period during the Covid-19 pandemic, it could be interesting to look at the lockdown dates in France, since bike trends should change during those times. We will use the package `lockdowndates` and merge it with our original data set.

These new features are designed to deepen our understanding of the factors influencing bike traffic, thereby enhancing the predictive power of our model.

In [20]:
# Create a new column 'weather' based on conditions
def categorize_weather(row, mean_rain, mean_snow, mean_windspeed):

    #Goal: Create a column weather with the categories: Clear (could also be cloudy), Windy, Rain, Snow
    #Consider the above if the value is greater than the mean value.
    if row['prcp'] > mean_rain:
        return 'Rainy'
    elif row['snow'] > mean_snow:
        return 'Snowy'
    elif row['wspd'] > mean_windspeed:
        return 'Windy'
    else:
        return 'Clear' #Note, we cannot differentiate whether it is sunny or cloudy. Just that the conditions above are not met

In [21]:
# Time of Day Category
def categorize_time_of_day(hour):
        if 6 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 18:
            return 'Afternoon'
        elif 18 <= hour < 24:
            return 'Evening'
        else:
            return 'Night'

In [22]:
def flag_rush_hour(hour):
    """
    Flags rush hour periods based on the hour of the day.

    Args:
    hour (int): Hour of the day (0-23).

    Returns:
    int: 1 if it's rush hour, otherwise 0.
    """
    # Define morning and evening rush hours (you can adjust these based on local patterns)
    morning_rush = (7, 8, 9)
    evening_rush = (16, 17, 18)

    if hour in morning_rush or hour in evening_rush:
        return 1
    else:
        return 0

In [23]:
def add_lockdown_curfew_features(df):
    """
    Add lockdown and curfew features to the DataFrame.
    
    Args:
    df (DataFrame): The input DataFrame.
    
    Returns:
    DataFrame: The DataFrame with lockdown and curfew features added.
    """
    # Reset the index to make 'date' a column if it's not already a column
    if isinstance(df.index, pd.DatetimeIndex):
        df.reset_index(inplace=True)

    # Get the start and end dates in string format
    start_date_str = df['date'].min().strftime('%Y-%m-%d')
    end_date_str = df['date'].max().strftime('%Y-%m-%d')

    # Initialize LockdownDates for France with the specified dates and restrictions
    ld = LockdownDates("France", start_date_str, end_date_str, ("stay_at_home", "masks"))
    lockdown_dates = ld.dates()

    # Check if the returned DataFrame from LockdownDates is empty
    if not lockdown_dates.empty:
        # Merge the lockdown information based on the date
        df = df.merge(lockdown_dates, left_on='date', right_index=True, how='left')

        df.drop(columns='france_country_code', inplace=True)

        # Fill NaN values that resulted from merge operation
        df['france_masks'].fillna(0, inplace=True)
        df['france_stay_at_home'].fillna(0, inplace=True)

        df['france_masks'] = df['france_masks'].astype(int)
        df['france_stay_at_home'] = df['france_stay_at_home'].astype(int)

        # Rename columns
        df.rename(columns={'france_masks': 'masks_code', 'france_stay_at_home': 'stay_at_home_code'}, inplace=True)

    else:
        # If no lockdown data is available, add default columns with 0
        df['france_masks'] = 0
        df['france_stay_at_home'] = 0

    return df

In [24]:
def create_new_features(df,mean_rain, mean_snow, mean_windspeed):
    """
    Create new features in the DataFrame.

    Args:
    df (DataFrame): The input DataFrame.

    Returns:
    DataFrame: The DataFrame with new features.
    """
    # Categorizing weather
    df['weather'] = df.apply(lambda row: categorize_weather(row, mean_rain, mean_snow, mean_windspeed), axis=1)

    # Categorizing time of the day
    df['time_of_day'] = df['hour'].apply(categorize_time_of_day)

    # Flagging rush hour
    df['rush_hour'] = df['hour'].apply(flag_rush_hour)
    
    # Seasonal Trends
    # TO BE DONE ? NYE, tour de france, events/nuits blanches/festivals etc.
    #df['high_season'] = df['month'].apply(lambda x: 1 if x in [6, 7, 8] else 0)  # Summer months as high season

    # Historical Traffic Patterns
    # This feature requires historical data
    # Consider previous bike data in paris ? require external past data
    # df['historical_avg'] = calculate_historical_avg(df)

    #other ideas:
    #Public Transport Strikes
    #Daylight Hours
    #COVID-19 Restrictions

    df = add_lockdown_curfew_features(df)
    
    return df

## Feature Transformation
In the feature transformation phase of our project, we focus on optimizing and tailoring our dataset for predictive modeling:

- **Dataset Segmentation**: 

    We introduce an innovative step to split the dataset based on the `working_day` feature. This allows us to separately analyze and model bike traffic patterns for working days and non-working days, recognizing that these two groups may exhibit distinct behaviors.

**Note:** We will apply `one-hot encoding` to categorical variables like `site_name`, `weather`, `month`, and `hour` when creating our pipeline for the machine learning model. 

These transformations are key to ensuring our model accurately captures the nuances of bike traffic patterns in different contexts.

In [25]:
def split_dataset_by_working_day(df):
    """
    Splits the dataset into two based on working day.

    Args:
    df (DataFrame): The input DataFrame.

    Returns:
    tuple: DataFrames split by working day.
    """
    # Splitting dataset
    df_working_day = df[df['working_day'] == 1]
    df_non_working_day = df[df['working_day'] == 0]

    df_working_day.drop(columns=['working_day'], inplace=True)
    df_non_working_day.drop(columns=['working_day'], inplace=True)
    
    return df_working_day, df_non_working_day

## Final prep and Conclusion

In the final part of our project, we bring together all feature engineering steps to prepare our dataset for model training and evaluation:

#### 1. Encoding Dates and Weather Data: 

We start by encoding date-related features and incorporating weather data into our dataset.

#### 2. Data Cleaning: 

Next, we clean the dataset, handling missing values and removing duplicates.

#### 3. Feature Creation and Transformation: 

We create new features, including weather categorization, and apply transformations like one-hot encoding.

#### 4. Data Splitting and Saving: 

Finally, we split the dataset based on working days and save the processed data for both training and testing phases.

In [26]:
def set_time_index(df):
    """
    Sets the 'date' column as the index of the DataFrame, if it exists.

    Args:
    df (DataFrame): The input DataFrame.

    Returns:
    DataFrame: The DataFrame with 'time' set as the index.
    """
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])  # Ensure 'date' is in datetime format
        df.set_index('date', inplace=True)
        print("Date column set as index.")
    else:
        print("Date column not found in DataFrame.")
    return df

In [27]:
def handle_missing_values(df, knn_flag=False, save_path=None, load_path=None):
    """
    Handles missing values in the DataFrame with specific strategies for each column.

    Args:
    df (DataFrame): The input DataFrame.
    knn_flag (bool): Flag to determine whether to perform KNN imputation.
    save_path (str): Path to save the DataFrame after KNN imputation.
    load_path (str): Path to load the DataFrame if KNN imputation is not performed.

    Returns:
    DataFrame: The DataFrame after handling missing values.
    """
    # Define columns that require specific imputation strategies
    zero_fill_cols = ['prcp']  # Assuming no precipitation for missing values

    # Fill with zeros
    for col in zero_fill_cols:
        if col in df.columns:
            df[col].fillna(0, inplace=True)

    # KNN Imputation for 'snow'
    if load_path == 'data/bike_df_imputed.parquet':
        # Load the DataFrame with the imputed values from a parquet file
        df['snow'] = pd.read_parquet(load_path)['snow']

    elif 'snow' in df.columns and knn_flag:
        # Perform KNN imputation
        knn_imputer = KNNImputer(n_neighbors=5)
        df[['snow']] = knn_imputer.fit_transform(df[['snow']].to_numpy().reshape(-1, 1))

        # Save the DataFrame with the imputed values, only if the file doesn't already exist
        if save_path and not os.path.exists(save_path):
            df.to_parquet(save_path)

    # Check and fill any remaining missing values for other columns
    for col in df.columns:
        if df[col].isna().any():
            # Choose a default imputation strategy for other columns (e.g., median)
            df[col].fillna(df[col].median(), inplace=True)

    return df

In [28]:
def remove_duplicates(df):
    """
    Remove duplicate rows from a DataFrame and provide information about the removal.

    Parameters:
    - df (pandas.DataFrame): The input DataFrame containing potentially duplicate rows.

    Returns:
    - pandas.DataFrame: A DataFrame with duplicate rows removed.
    """
    initial_row_count = df.shape[0]
    df = df.drop_duplicates()
    final_row_count = df.shape[0]

    print(f"Removed {initial_row_count - final_row_count} duplicates. Remaining rows: {final_row_count}")
    return df

In [36]:
def clean_data(df, visualization=False):
    """
    Cleans the DataFrame by handling missing values, removing duplicates, setting the time index, 
    and dropping redundant or low correlation columns.

    Args:
    df (DataFrame): The input DataFrame.

    Returns:
    DataFrame: Cleaned DataFrame.
    """
    # Handling missing values
    # If performing KNN imputation and saving the result (otherwise, set knn_flag = False)
    df = handle_missing_values(df, knn_flag=True, save_path='data/bike_df_imputed.parquet', load_path='data/bike_df_imputed.parquet')

    # Removing duplicates
    df = remove_duplicates(df)

    # Setting the time index
    df = set_time_index(df)

    # List of columns to drop

    if visualization == True:
        # We are keeping more features for visualization purposes, even if in our prediction model, we will not use them
        drop_columns = [
            'site_name', 'site_id', 'coordinates', 'latitude', 'longitude', 'tsun', 
            'counter_id', 'counter_installation_date', 'counter_technical_id', 'date'
        ]

    else:
        drop_columns = [
            'counter_name', 'site_id', 'coordinates', 'bike_count', 'latitude', 'longitude', 'year', 
            'day', 'tsun', 'weekday', 'coco', 'pres', 'wspd', 'wdir', 'dwpt', 'date',
            'counter_id', 'counter_installation_date', 'counter_technical_id'
        ]
    
    # Only drop columns that are present in the DataFrame
    columns_to_drop = [col for col in drop_columns if col in df.columns]
    df = df.drop(columns_to_drop, axis=1)

    return df

In [30]:
def feature_engineering(df, visualization=False):
    """
    Apply all feature engineering steps to the given DataFrame.

    Args:
    df (DataFrame): The input DataFrame.

    Returns:
    DataFrame: The DataFrame after feature engineering.
    """

    # Apply date encoding
    df = encode_dates(df)
    
    # Incorporate weather data
    df = get_weather_data(df)

    # Calculate mean values for each weather column before cleaning
    mean_rain = df['prcp'].mean()
    mean_snow = df['snow'].mean()
    mean_windspeed = df['wspd'].mean()

    # Create new features (including weather category using the calculated means)
    df = create_new_features(df, mean_rain, mean_snow, mean_windspeed)

    # Clean the data (handle missing values, remove duplicates, etc.)
    df = clean_data(df, visualization)

    return df

In [37]:
# Usage
bike_df_train = pd.read_parquet(Path("data") / "train.parquet")
bike_df_test = pd.read_parquet(Path("data") / "test.parquet")

train_processed_viz = feature_engineering(bike_df_train, visualization=True)
test_processed_viz = feature_engineering(bike_df_test, visualization=True)

train_processed = feature_engineering(bike_df_train)
test_processed = feature_engineering(bike_df_test)

Fetching lockdown dates...
Fetched lockdown dates for: France
Removed 0 duplicates. Remaining rows: 496827
Date column set as index.
Fetching lockdown dates...
Fetched lockdown dates for: France
Removed 0 duplicates. Remaining rows: 41608
Date column set as index.
Fetching lockdown dates...
Fetched lockdown dates for: France
Removed 0 duplicates. Remaining rows: 496827
Date column set as index.
Fetching lockdown dates...
Fetched lockdown dates for: France
Removed 0 duplicates. Remaining rows: 41608
Date column set as index.


In [38]:
def align_datasets(train_df, test_df, save_train_path=None, save_test_path=None):
    # Combine columns from both datasets
    all_columns = set(train_df.columns).union(set(test_df.columns))

    # Reindex both datasets to have the same columns, fill missing with 0
    train_df_aligned = train_df.reindex(columns=all_columns, fill_value=0)
    test_df_aligned = test_df.reindex(columns=all_columns, fill_value=0)

    # Optionally save the aligned datasets
    if save_train_path:
        train_df_aligned.to_parquet(save_train_path)
    if save_test_path:
        test_df_aligned.to_parquet(save_test_path)

    return train_df_aligned, test_df_aligned

In [39]:
def save_aligned_processed_data(train_df, test_df, filename, split_by_working_day=True):
    # Align datasets
    train_aligned, test_aligned = align_datasets(train_df, test_df)

    # Process for working and non-working day datasets
    if split_by_working_day:
        for df, part in [(train_aligned, "train"), (test_aligned, "test")]:
            df_working_day, df_non_working_day = split_dataset_by_working_day(df)
            df_working_day.to_parquet(Path("data") / f"{part}_{filename}_working_day.parquet")
            df_non_working_day.to_parquet(Path("data") / f"{part}_{filename}_non_working_day.parquet")
    else:
        # Save aligned datasets without splitting
        train_aligned.to_parquet(Path("data") / f"train_{filename}.parquet")
        test_aligned.to_parquet(Path("data") / f"test_{filename}.parquet")

In [40]:
# Don't split for visualization
save_aligned_processed_data(train_processed_viz, test_processed_viz, "processed_viz", split_by_working_day=False)

# Split for modeling
save_aligned_processed_data(train_processed, test_processed, "processed")

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Approach and Challenges:

We anticipate challenges in ensuring feature relevance and avoiding overfitting. We will tackle these through careful feature selection and validation, ensuring our model remains generalizable and robust.

# Conclusion:

This notebook is critical for setting up a data foundation that’s not only rich in information but also tailored for effective modeling in the next stages. The quality and ingenuity of the features engineered here will be a determinant factor in the accuracy of our predictions.

Now that we added a lot of new features, we can go back into the EDA stage (`part 3 - post processing visualization`) to justify our approach on feature engineering.